In [1]:
##LEGO DATA DENORMALIZATION - RACHAEL BURNS
import csv

with open("inventory_parts.csv", "rt") as inventory_parts:
    reader = csv.DictReader(inventory_parts)
    headers = reader.fieldnames
    ivty = {}
    i=-1
    for header in headers:
        ivty[header]={}
    for row in reader:
        i+=1
        for header in headers:
            ivty[header][i]=row[header]  
        
    most = i+1
    
key_list = [item for item in ivty["color_id"].keys()]
print(key_list[0:5])
#print([header + " " + str(type(ivty[header]))+str(len(ivty[header])) for header in headers])

with open("colors.csv", "rt") as colors:
    reader = csv.DictReader(colors)
    headers = {"color_name":"name", "color_rgb":"rgb", "color_istrans":"is_trans"}
    color_data = {}
    for header in headers.keys():
        ivty[header] = {}
    for row in reader:
        color_data[int(row["id"])]=row
    for header in headers.keys():
        for i in range(0,most):
            ivty[header][i] = color_data[int(ivty["color_id"][i])][headers[header]]

#print([key + " " + ivty[key][i] for key in ivty.keys() for i in range(0,5)])


with open("inventories.csv") as parts:
    reader = csv.DictReader(parts)
    headers = {"ivty_version":"version", "set_num":"set_num"}
    ivty_data = {}
    for header in headers.keys():
        ivty[header] = {}
    for row in reader:
        ivty_data[row["id"]]=row
    for header in headers.keys():
        for i in range(0,most):
            ivty[header][i] = ivty_data[ivty["inventory_id"][i]][headers[header]]


[0, 1, 2, 3, 4]
364


In [18]:
import re

with open("parts.csv") as parts:
    count_exception = 0
    reader = csv.DictReader(parts)
    headers = {"part_name":"name", "part_cat_id":"part_cat_id"}
    part_data = {}
    for header in headers.keys():
        ivty[header] = {}
    for row in reader:
        part_data[row["part_num"]]=row
    for header in headers.keys():
        for i in range(0,most):
            try:
                ivty[header][i] = part_data[ivty["part_num"][i]][headers[header]]
            except:
                #***NOTE- 364 keys do not match - ex/ 48002 in inventory parts table corresponds to 
                #48002a, 480002b in inventory table. This is bad DB design!
                #Need to implement some kind of regexp match - placeholder for now. 
                pattern = re.compile(ivty["part_num"][i] + "[a-z]?")
                match_name = ""
                for key in part_data.keys():
                    if pattern.match(key):
                        if match_name:
                            match_name += " / " 
                        match_name += part_data[key]["name"] 
                        ivty["part_cat_id"][i] = part_data[key]["part_cat_id"]
                ivty["part_name"][i] = match_name


In [78]:
#print([ivty[key].keys() for key in ivty.keys()])
#print([key + " " + ivty[key][i] for key in ivty.keys() for i in range(0, 10)])

In [3]:
with open("inventory_sets.csv") as ivty_sets:
    reader = csv.DictReader(ivty_sets)
    headers = {"set_quantity":"quantity"}
    ivty_set_data = {}
    count_excp=0
    for header in headers.keys():
        ivty[header] = {}
    for row in reader:
        ivty_set_data[row["set_num"]]=row
    for header in headers.keys():
        for i in range(0,most):
            try:
                ivty[header][i] = ivty_set_data[ivty["set_num"][i]][headers[header]]
            except:
                #***NOTE: most sets are not in the inventory_sets table, so not all rows will
                #have a quantity - for this reason and also because there's not a clear definition of
                #either "inventory" or "quantity" I would recommend disregarding the quanity column.
                count_excp+=1
                
print(count_excp)

501882


In [42]:
#NOTE: DO NOT RUN AGAIN! Will overwrite file with empty rows. 
#NOTE: Source file for sets has non-ascii characters that raise an error when trying to read as csv.

with open('sets.csv', 'rb') as file:
    with open('sets_out.csv', 'wt') as file2:
        for row in file:
            for char in row:
                if char<=128:
                    try:
                        file2.write(chr(char))
                    except:
                        pass
#This script creates a blank row between each row, I removed in excel. 

In [4]:
with open("sets_out.csv", 'rt') as sets:
    reader = csv.DictReader(sets) 
    headers = {"set_name":"name", "year":"year", "theme_id":"theme_id", "set_parts_count":"num_parts"}
    set_data = {}
    count_excp=0
    excp_list = []
    for header in headers.keys():
        ivty[header] = {}
    for row in reader:
        set_data[row["set_num"]]=row
    for header in headers.keys():
        for i in range(0,most):
            try:
                ivty[header][i] = set_data[ivty["set_num"][i]][headers[header]]
            except Exception as e:
                #if e not in excp_list:
                #    excp_list.append(e)
                count_excp+=1
print(count_excp)

#***NOTE - not all set ids in inventory file are in the sets file This is only 2796 instances There
#does not seem to be a pattern as there was with the part names. This file also has the year, which 
#is important to many of our questions - 
#it makes sense to disregard the rows that do not have a year populated for that part of the analysis.



2796


In [19]:
with open("themes.csv", 'rt') as themes:
    reader = csv.DictReader(themes) 
    headers = {"theme_name":"name", "theme_parent_id": "parent_id"}
    theme_data = {}
    count_excp=0
    excp_list = []
    for header in headers.keys():
        ivty[header] = {}
    for row in reader:
        theme_data[row["id"]]=row
    for header in headers.keys():
        for i in range(0,most):
            try:
                ivty[header][i] = theme_data[ivty["theme_id"][i]][headers[header]]
            except Exception as e:
                #**NOTE: of the exceptions, 699 are no theme id and 699 are theme id not in theme table.
                excp_list.append(e)
                count_excp+=1
    ivty["parent_name"] = {}
    for key in ivty["theme_parent_id"].keys():
        try:
            ivty["parent_name"][key] = theme_data[ivty["theme_parent_id"][key]]["name"]
            #**NOTE: Next two lines check if a parent theme has another parent - some do, do we want to 
            #capture that in our data?
            #if len(theme_data[ivty["theme_parent_id"][key]]["parent_id"])>0:
            #    print(theme_data[ivty["theme_parent_id"][key]]["parent_id"])
        except:
            #exceptions are themes that do not have a parent, which is fine.
            pass
print(count_excp)
#print(excp_list)
#print([key + " " + ivty[key][i] for key in ivty.keys() for i in range(0, 10) if i in ivty[key].keys()])
print(580251-579552)
print(1398-699)

with open("part_categories.csv", 'rt') as part_cats:
    reader = csv.DictReader(part_cats) 
    headers = {"part_cat_name":"name"}
    part_cat_data = {}
    count_excp=0
    excp_list = []
    for header in headers.keys():
        ivty[header] = {}
    for row in reader:
        set_data[row["id"]]=row
    for header in headers.keys():
        for i in range(0,most):
            try:
                ivty[header][i] = set_data[ivty["part_cat_id"][i]][headers[header]]
            except Exception as e:
                #182 parts are not in a category.
                if e not in excp_list:
                    excp_list.append(e)
                count_excp+=1
print(count_excp)
#print(excp_list)

print("\nRow , Num Populated")
for key in ivty.keys():
        print(key, ",", len(ivty[key].keys()))

1398
699
699
107

Row , Num Populated
inventory_id , 580251
part_num , 580251
color_id , 580251
quantity , 580251
is_spare , 580251
color_name , 580251
color_rgb , 580251
color_istrans , 580251
part_name , 580251
part_cat_id , 580144
ivty_version , 580251
set_num , 580251
set_quantity , 78369
set_name , 579552
year , 579552
theme_id , 579552
set_parts_count , 579552
theme_name , 579552
theme_parent_id , 579552
parent_name , 371970
part_cat_name , 580144


In [39]:
with open("lego_out.csv", 'wt') as out:
    writer = csv.DictWriter(out, ivty.keys(), dialect = 'excel', lineterminator = "\n")
    writer.writeheader()
    for i in range(0, 580251):
        row = {}
        for key in ivty.keys():
            try:
                row[key] = ivty[key][i]
            except: 
                row[key] = ""
        writer.writerow(row)
        
print("Finished!")

Finished!


There is probably a more compact and higher-performance way to implement these joins - eg. I know some python packages mimic SQL joins. However doing it from scratch was a good exercise because it made visible the inconsistencies in the data.